In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler

# ✅ Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [8]:
# Cell 2: Load and Prepare Data
df = pd.read_csv("IL_T700_cleaned.csv")

# Check and drop NaN values
df.dropna(inplace=True)

# Separate features (X) and label (y)
X = df.drop(columns=['label']).values
y = df['label'].values

# Balance the dataset with RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(f"Balanced dataset: {np.bincount(y_resampled)}")

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_resampled)


Balanced dataset: [10000 10000]


In [9]:
# Cell 3: Create Sequences for LSTM
timesteps = 30  # Adjusted sequence length

def create_sequences(data, labels, seq_len=30):
    seqs, labs = [], []
    for i in range(len(data) - seq_len + 1):
        seqs.append(data[i : i + seq_len])
        labs.append(labels[i + seq_len - 1])
    return np.array(seqs), np.array(labs)

X_seq, y_seq = create_sequences(X_scaled, y_resampled, seq_len=timesteps)

print("Sequence X_seq shape:", X_seq.shape)
print("Sequence y_seq shape:", y_seq.shape)


Sequence X_seq shape: (19971, 30, 19)
Sequence y_seq shape: (19971,)


In [10]:
# Cell 4: TimeSeriesSplit for Train-Test Split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X_seq):
    X_train, X_test = X_seq[train_index], X_seq[test_index]
    y_train, y_test = y_seq[train_index], y_seq[test_index]

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape, "y_test shape:", y_test.shape)


X_train shape: (16643, 30, 19) X_test shape: (3328, 30, 19)
y_train shape: (16643,) y_test shape: (3328,)


In [11]:
# Cell 5: Prepare Tensors and DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test,  dtype=torch.long).to(device)

batch_size = 32  # Reduced batch size for better gradient updates
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [12]:
# Cell 6: Define LSTM Model
class DeeperLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, num_layers=2, dropout=0.5):
        super(DeeperLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim // 2, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_step = lstm_out[:, -1, :]
        x = self.fc1(last_step)
        x = self.relu(x)
        out = self.fc2(x)
        return out

num_features = X.shape[1]
model = DeeperLSTM(input_dim=num_features).to(device)


In [13]:
# Cell 7: Define Weighted Loss and Optimizer
class_weights = torch.tensor([0.5, 1.5]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)  # Added L2 regularization

print("Criterion:", criterion)
print("Optimizer:", optimizer)


Criterion: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.0001
)


In [14]:
# Cell 8: Train the Model
epochs = 100
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += yb.size(0)
        correct += (predicted == yb).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = 100.0 * correct / total
    print(f"Epoch {epoch}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")


Epoch 1/100 | Loss: 0.4209 | Acc: 67.03%
Epoch 2/100 | Loss: 0.0757 | Acc: 98.19%
Epoch 3/100 | Loss: 0.0609 | Acc: 98.59%
Epoch 4/100 | Loss: 0.0518 | Acc: 98.87%
Epoch 5/100 | Loss: 0.0523 | Acc: 98.76%
Epoch 6/100 | Loss: 0.0507 | Acc: 98.84%
Epoch 7/100 | Loss: 0.0359 | Acc: 99.32%
Epoch 8/100 | Loss: 0.0371 | Acc: 99.23%
Epoch 9/100 | Loss: 0.0343 | Acc: 99.38%
Epoch 10/100 | Loss: 0.0383 | Acc: 99.33%
Epoch 11/100 | Loss: 0.0485 | Acc: 98.94%
Epoch 12/100 | Loss: 0.0360 | Acc: 99.33%
Epoch 13/100 | Loss: 0.0287 | Acc: 99.53%
Epoch 14/100 | Loss: 0.0312 | Acc: 99.48%
Epoch 15/100 | Loss: 0.0332 | Acc: 99.37%
Epoch 16/100 | Loss: 0.0284 | Acc: 99.55%
Epoch 17/100 | Loss: 0.0354 | Acc: 99.51%
Epoch 18/100 | Loss: 0.0268 | Acc: 99.56%
Epoch 19/100 | Loss: 0.0285 | Acc: 99.52%
Epoch 20/100 | Loss: 0.0289 | Acc: 99.54%
Epoch 21/100 | Loss: 0.0305 | Acc: 99.46%
Epoch 22/100 | Loss: 0.0302 | Acc: 99.48%
Epoch 23/100 | Loss: 0.0245 | Acc: 99.57%
Epoch 24/100 | Loss: 0.0307 | Acc: 99.49%
E

In [15]:
# Cell 9: Evaluate the Model
model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for Xb, yb in test_loader:
        logits = model(Xb)
        _, predicted = torch.max(logits, 1)
        all_preds.append(predicted.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true = np.concatenate(all_true)

acc  = accuracy_score(all_true, all_preds) * 100
prec = precision_score(all_true, all_preds, pos_label=1, zero_division=0) * 100
rec  = recall_score(all_true, all_preds, pos_label=1, zero_division=0) * 100
f1   = f1_score(all_true, all_preds, pos_label=1, zero_division=0) * 100

print("\n✅ Final Test Results:")
print(f"Accuracy:   {acc:.2f}%")
print(f"Precision:  {prec:.2f}%")
print(f"Recall:     {rec:.2f}%")
print(f"F1-score:   {f1:.2f}%")



✅ Final Test Results:
Accuracy:   99.70%
Precision:  100.00%
Recall:     99.70%
F1-score:   99.85%


In [16]:
df = pd.read_csv("IL_T700_cleaned.csv")
df.dropna(inplace=True)

X = df.drop(columns=['label']).values
y = df['label'].values

# Balance the dataset with RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Data Augmentation for anomalies
anomalies = X_resampled[y_resampled == 1]
augmented_anomalies = anomalies + np.random.normal(0, 0.01, anomalies.shape)
X_resampled = np.vstack((X_resampled, augmented_anomalies))
y_resampled = np.hstack((y_resampled, np.ones(len(augmented_anomalies), dtype=int)))
y_resampled = y_resampled.astype(int)  # Ensure all labels are integers

print(f"Balanced and Augmented dataset: {np.bincount(y_resampled)}")

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_resampled)


Balanced and Augmented dataset: [10000 20000]


In [17]:
timesteps = 30

def create_sequences(data, labels, seq_len=30):
    seqs, labs = [], []
    for i in range(len(data) - seq_len + 1):
        seqs.append(data[i : i + seq_len])
        labs.append(labels[i + seq_len - 1])
    return np.array(seqs), np.array(labs)

X_seq, y_seq = create_sequences(X_scaled, y_resampled, seq_len=timesteps)


In [18]:
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X_seq):
    X_train, X_test = X_seq[train_index], X_seq[test_index]
    y_train, y_test = y_seq[train_index], y_seq[test_index]

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape, "y_test shape:", y_test.shape)


X_train shape: (24976, 30, 19) X_test shape: (4995, 30, 19)
y_train shape: (24976,) y_test shape: (4995,)


In [19]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test,  dtype=torch.long).to(device)

batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [20]:
class DeeperLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, num_layers=2, dropout=0.5):
        super(DeeperLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim // 2, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_step = lstm_out[:, -1, :]
        x = self.fc1(last_step)
        x = self.relu(x)
        out = self.fc2(x)
        return out

num_features = X.shape[1]
model = DeeperLSTM(input_dim=num_features).to(device)


In [21]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

criterion = FocalLoss(alpha=0.25, gamma=2)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


/home/spuspa/.local/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [22]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

criterion = FocalLoss(alpha=0.25, gamma=2)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


In [23]:
epochs = 100
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += yb.size(0)
        correct += (predicted == yb).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = 100.0 * correct / total
    scheduler.step(epoch_loss)
    print(f"Epoch {epoch}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")


Epoch 1/100 | Loss: 0.0163 | Acc: 85.73%
Epoch 2/100 | Loss: 0.0021 | Acc: 99.17%
Epoch 3/100 | Loss: 0.0019 | Acc: 99.18%
Epoch 4/100 | Loss: 0.0014 | Acc: 99.40%
Epoch 5/100 | Loss: 0.0014 | Acc: 99.41%
Epoch 6/100 | Loss: 0.0012 | Acc: 99.48%
Epoch 7/100 | Loss: 0.0009 | Acc: 99.64%
Epoch 8/100 | Loss: 0.0009 | Acc: 99.67%
Epoch 9/100 | Loss: 0.0011 | Acc: 99.57%
Epoch 10/100 | Loss: 0.0009 | Acc: 99.65%
Epoch 11/100 | Loss: 0.0009 | Acc: 99.60%
Epoch 12/100 | Loss: 0.0008 | Acc: 99.69%
Epoch 13/100 | Loss: 0.0010 | Acc: 99.66%
Epoch 14/100 | Loss: 0.0009 | Acc: 99.69%
Epoch 15/100 | Loss: 0.0008 | Acc: 99.66%
Epoch 16/100 | Loss: 0.0008 | Acc: 99.68%
Epoch 17/100 | Loss: 0.0009 | Acc: 99.58%
Epoch 18/100 | Loss: 0.0008 | Acc: 99.66%
Epoch 19/100 | Loss: 0.0007 | Acc: 99.78%
Epoch 20/100 | Loss: 0.0008 | Acc: 99.64%
Epoch 21/100 | Loss: 0.0008 | Acc: 99.70%
Epoch 22/100 | Loss: 0.0008 | Acc: 99.66%
Epoch 23/100 | Loss: 0.0008 | Acc: 99.70%
Epoch 24/100 | Loss: 0.0007 | Acc: 99.74%
E

In [24]:
model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for Xb, yb in test_loader:
        logits = model(Xb)
        _, predicted = torch.max(logits, 1)
        all_preds.append(predicted.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true = np.concatenate(all_true)

acc  = accuracy_score(all_true, all_preds) * 100
prec = precision_score(all_true, all_preds, pos_label=1) * 100
rec  = recall_score(all_true, all_preds, pos_label=1) * 100
f1   = f1_score(all_true, all_preds, pos_label=1) * 100

print("\n✅ Final Test Results:")
print(f"Accuracy:   {acc:.2f}%")
print(f"Precision:  {prec:.2f}%")
print(f"Recall:     {rec:.2f}%")
print(f"F1-score:   {f1:.2f}%")



✅ Final Test Results:
Accuracy:   99.72%
Precision:  100.00%
Recall:     99.72%
F1-score:   99.86%
